In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [2]:
f = open('all_outputs_train.pkl','rb')
all_outputs_train = pickle.load(f)

In [3]:
f.close()

In [4]:
g = open('all_outputs_val.pkl','rb')
all_outputs_val = pickle.load(g)
g.close()

In [5]:
class Attention(nn.Module):
    def __init__(self, pre_trained_net):
        super(Attention, self).__init__()
        self.L = 1000
        self.D = 128
        self.K = 1
        self.pre_trained_net = pre_trained_net
        self.l1 = nn.Sequential(nn.Linear(self.L, self.D), nn.Tanh())
        self.l2 = nn.Sequential(nn.Linear(self.L, self.D), nn.Sigmoid())
        self.l3 = nn.Linear(self.D, self.K)
        self.classifier = nn.Sequential(nn.Linear(self.L * self.K, self.L * self.K),nn.ReLU())
        self.final = nn.Sequential(nn.Linear(self.L * self.K,10),nn.Softmax(dim=1))
        self.classifier_debug = nn.Sequential(nn.Linear(self.L, 10), nn.Softmax(dim=1))

    def forward(self, x):
        H = self.pre_trained_net(x)
        A_V = self.l1(H)
        A_U = self.l2(H)
        A = self.l3(A_V * A_U)
        A = torch.transpose(A, 1, 0)
        A = F.softmax(A, dim=1)
        M = torch.mm(A, H)
        Y = self.classifier(M)
        Y = self.final(Y)
        return Y

In [6]:
pre_trained_net = torchvision.models.resnet18(pretrained=True)
model = Attention(pre_trained_net)

In [7]:
model.load_state_dict(torch.load('trained_model.pth'))
model = model.cuda()

In [8]:
all_outputs_train.keys()

dict_keys(['ZT111_4_A_1_12', 'ZT111_4_A_1_13', 'ZT111_4_A_1_14', 'ZT111_4_A_1_2', 'ZT111_4_A_1_5', 'ZT111_4_A_1_6', 'ZT111_4_A_1_7', 'ZT111_4_A_1_8', 'ZT111_4_A_1_9', 'ZT111_4_A_2_1', 'ZT111_4_A_2_10', 'ZT111_4_A_2_13', 'ZT111_4_A_2_14', 'ZT111_4_A_2_2', 'ZT111_4_A_2_8', 'ZT111_4_A_3_1', 'ZT111_4_A_3_10', 'ZT111_4_A_3_11', 'ZT111_4_A_3_12', 'ZT111_4_A_3_13', 'ZT111_4_A_3_2', 'ZT111_4_A_3_4', 'ZT111_4_A_3_5', 'ZT111_4_A_3_7', 'ZT111_4_A_3_8', 'ZT111_4_A_3_9', 'ZT111_4_A_4_11', 'ZT111_4_A_4_13', 'ZT111_4_A_4_14', 'ZT111_4_A_4_2', 'ZT111_4_A_4_4', 'ZT111_4_A_4_5', 'ZT111_4_A_4_6', 'ZT111_4_A_4_8', 'ZT111_4_A_4_9', 'ZT111_4_A_5_10', 'ZT111_4_A_5_11', 'ZT111_4_A_5_12', 'ZT111_4_A_5_13', 'ZT111_4_A_5_14', 'ZT111_4_A_5_3', 'ZT111_4_A_5_5', 'ZT111_4_A_5_9', 'ZT111_4_A_6_10', 'ZT111_4_A_6_11', 'ZT111_4_A_6_12', 'ZT111_4_A_6_13', 'ZT111_4_A_6_14', 'ZT111_4_A_6_4', 'ZT111_4_A_6_5', 'ZT111_4_A_6_6', 'ZT111_4_A_6_7', 'ZT111_4_A_6_8', 'ZT111_4_A_6_9', 'ZT111_4_A_7_1', 'ZT111_4_A_7_10', 'ZT111_4_A_7_

In [9]:
type(all_outputs_train.values())

dict_values

In [10]:
import os
import pandas as pd
def preprocess(image):
    transform = transforms.Compose([
        # transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        # transforms.RandomCrop([224, 224]),
        transforms.Resize(224,interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
    ])
    return transform(image)


# encoding: [0+0,3+3,4+4,5+5,3+4,3+5,4+3,4+5,5+3,5+4]

def onehot(file):
    filew = pd.read_csv(file)
    # one_hots = np.empty((filew.shape[0], 11),dtype=object)
    one_hots = {}
    for i in range(filew.shape[0]):
        primary_score = filew.loc[i][0][-3]
        secondary_score = filew.loc[i][0][-1]
        # one_hots[i,0] = filew.loc[i][0][:-4]
        key = filew.loc[i][0][:-4]
        if primary_score == '0' and secondary_score == '0':
            # one_hots[i, 1:] = np.eye(10)[0, :]
            one_hots[key] = np.eye(10)[0,:]
        elif primary_score == '1' and secondary_score == '1':
            # one_hots[i, 1:] = np.eye(10)[1, :]
            one_hots[key] = np.eye(10)[1, :]
        elif primary_score == '2' and secondary_score == '2':
            # one_hots[i, 1:] = np.eye(10)[2, :]
            one_hots[key] = np.eye(10)[2, :]
        elif primary_score == '3' and secondary_score == '3':
            # one_hots[i, 1:] = np.eye(10)[3, :]
            one_hots[key] = np.eye(10)[3, :]
        elif primary_score == '1' and secondary_score == '2':
            # one_hots[i, 1:] = np.eye(10)[4, :]
            one_hots[key] = np.eye(10)[4, :]
        elif primary_score == '1' and secondary_score == '3':
            # one_hots[i, 1:] = np.eye(10)[5, :]
            one_hots[key] = np.eye(10)[5, :]
        elif primary_score == '2' and secondary_score == '1':
            # one_hots[i, 1:] = np.eye(10)[6, :]
            one_hots[key] = np.eye(10)[6, :]
        elif primary_score == '2' and secondary_score == '3':
            # one_hots[i, 1:] = np.eye(10)[7, :]
            one_hots[key] = np.eye(10)[7, :]
        elif primary_score == '3' and secondary_score == '1':
            # one_hots[i, 1:] = np.eye(10)[8, :]
            one_hots[key] = np.eye(10)[8, :]
        elif primary_score == '3' and secondary_score == '2':
            # one_hots[i, 1:] = np.eye(10)[9, :]
            one_hots[key] = np.eye(10)[9, :]
        else:
            print('you are wrong, there are more than 10 possibilities,primary score:{},secondary score:{}'.format(primary_score,secondary_score))
            # one_hots[i, :] = np.zeros(10)[0, :]
            one_hots[key] = np.eye(10)[0, :]
    return one_hots


PARENT_DIR = 'dataset_TMA'
train_val_masks_dir = os.path.join(PARENT_DIR, 'train_validation_patches_750')
train_dirs = []
val_dirs = []
train_dirs_net = []
val_dirs_net = []
for dir in os.listdir(train_val_masks_dir):
    if (dir.startswith('ZT111') or dir.startswith('ZT199') or dir.startswith('ZT204')):
        train_dirs.append(dir)
    elif dir.startswith('ZT76'):
        val_dirs.append(dir)
    if len(os.listdir(os.path.join(train_val_masks_dir,dir))) != 0:
        if (dir.startswith('ZT111') or dir.startswith('ZT199') or dir.startswith('ZT204')):
            train_dirs_net.append(dir)
        elif dir.startswith('ZT76'):
            val_dirs_net.append(dir)

tma_info = os.path.join(PARENT_DIR, 'tma_info')
train_arrays = ['ZT111', 'ZT199', 'ZT204']
val_arrays = ['ZT76']
one_hots_train = {}
one_hots_val = {}

for file in os.listdir(tma_info):
    if file[0:5] in train_arrays:
        one_hots_train[file[0:5]] = onehot(os.path.join(tma_info, file))
    if file[0:4] in val_arrays:
        one_hots_val[file[0:4]] = onehot(os.path.join(tma_info,file))

In [11]:
empty_train = []
empty_val = []
for idx,dir in enumerate(train_dirs):
    paths = os.path.join(train_val_masks_dir,dir)
    if len(os.listdir(paths)) == 0:
        empty_train.append(paths.split('\\')[2])
for idx,dir in enumerate(val_dirs):
    paths = os.path.join(train_val_masks_dir,dir)
    if len(os.listdir(paths)) == 0:
        empty_val.append(paths.split('\\')[2])
one_hots_val_processed = {}
one_hots_train_processed = {}
for array in train_arrays:
    dictionary = one_hots_train[array]
    one_hots_train_processed[array] = {}
    for key in dictionary.keys():
        if key not in empty_train:
            one_hots_train_processed[array][key] = dictionary[key]
for array in val_arrays:
    dictionary = one_hots_val[array]
    one_hots_val_processed[array] = {}
    for key in dictionary.keys():
        if key not in empty_val:
            one_hots_val_processed[array][key] = dictionary[key]

In [12]:
from tqdm import tqdm

In [15]:
len(one_hots_train_processed['ZT111'].keys())

225

In [16]:
225+176+100

501

In [18]:
len(train_dirs_net)

501

In [19]:
empty_train

['ZT111_4_A_5_7',
 'ZT111_4_B_8_2',
 'ZT204_6_A_1_11',
 'ZT204_6_A_6_2',
 'ZT204_6_B_1_3',
 'ZT204_6_B_2_5',
 'ZT204_6_B_2_9']

In [20]:
empty_val

['ZT76_39_A_2_9',
 'ZT76_39_A_5_4',
 'ZT76_39_A_7_4',
 'ZT76_39_A_7_8',
 'ZT76_39_B_1_4',
 'ZT76_39_B_6_6']

In [21]:
len(all_outputs_train)

501

In [22]:
all_outputs_train.keys() == one_hots_train_processed.keys()

False

In [23]:
one_hots_train_processed.keys()

dict_keys(['ZT111', 'ZT199', 'ZT204'])

In [30]:
keys = []
for key in one_hots_train_processed.keys():
    for k1 in one_hots_train_processed[key].keys():
        keys.append(k1)
len(keys)

501

In [31]:
all_outputs_train.keys() == keys

False

In [32]:
len(all_outputs_train.keys())

501

In [33]:
len(keys)

501

In [34]:
all_outputs_train[keys[0]]

tensor([[0.0019, 0.8085, 0.1806, 0.0019, 0.0018, 0.0010, 0.0012, 0.0012, 0.0009,
         0.0011]], requires_grad=True)

In [35]:
keys[:10]

['ZT111_4_A_1_12',
 'ZT111_4_A_1_13',
 'ZT111_4_A_1_14',
 'ZT111_4_A_1_2',
 'ZT111_4_A_1_5',
 'ZT111_4_A_1_6',
 'ZT111_4_A_1_7',
 'ZT111_4_A_1_8',
 'ZT111_4_A_1_9',
 'ZT111_4_A_2_1']

In [38]:
a = list(all_outputs_train.keys())
a[:10]

['ZT111_4_A_1_12',
 'ZT111_4_A_1_13',
 'ZT111_4_A_1_14',
 'ZT111_4_A_1_2',
 'ZT111_4_A_1_5',
 'ZT111_4_A_1_6',
 'ZT111_4_A_1_7',
 'ZT111_4_A_1_8',
 'ZT111_4_A_1_9',
 'ZT111_4_A_2_1']

In [39]:
a == keys

True

In [42]:
targets = {}
for key in one_hots_train_processed.keys():
    for k1,v1 in one_hots_train_processed[key].items():
        targets[k1] = v1
len(targets)

501

In [44]:
targets.keys() == all_outputs_train.keys()

True

In [61]:
counts = 0
for key in all_outputs_train.keys():
    target = np.argmax(targets[key])
    pred = torch.argmax(all_outputs_train[key])
    print(pred.item(),target)
    if target == pred:
        counts += 1
print(counts / len(targets.keys()))

1 4
1 1
1 1
1 1
1 1
1 1
1 4
1 2
1 1
1 4
1 1
1 2
1 1
1 2
1 2
1 2
1 2
1 2
1 4
1 2
1 7
1 7
1 2
1 6
1 1
1 1
1 3
1 4
1 1
1 9
1 1
1 7
1 1
1 2
1 2
1 1
1 4
1 2
1 1
1 2
1 1
1 1
1 4
1 7
1 2
1 2
1 2
1 1
1 4
1 1
1 1
1 1
1 3
1 4
1 1
1 7
1 5
1 1
1 1
1 1
1 7
1 9
1 1
1 2
1 3
1 6
1 2
1 1
1 4
1 2
1 2
1 2
1 1
1 7
1 2
1 7
1 9
1 1
1 1
1 2
1 2
1 1
1 1
1 4
1 2
1 2
1 1
1 4
1 4
1 9
1 4
1 4
1 2
1 1
1 2
1 4
1 1
1 2
1 1
1 1
1 4
1 1
1 7
1 1
1 2
1 2
1 1
1 2
1 1
1 1
1 4
1 1
1 2
1 1
1 3
1 2
1 1
1 9
1 1
1 4
1 2
1 1
1 3
1 3
1 1
1 4
1 7
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 4
1 9
1 1
1 2
1 1
1 1
1 2
1 8
1 2
1 4
1 6
1 1
1 1
1 8
1 4
1 3
1 1
1 6
1 4
1 1
1 1
1 4
1 4
1 4
1 6
1 2
1 6
1 2
1 2
1 2
1 2
1 1
1 4
1 2
1 1
1 2
1 2
1 1
1 2
1 1
1 1
1 3
1 2
1 1
1 1
1 1
1 1
1 1
1 2
1 4
1 2
1 2
1 1
1 1
1 2
1 1
1 1
1 2
1 1
1 2
1 4
1 2
1 1
1 4
1 4
1 1
1 4
1 2
1 1
1 1
1 2
1 1
1 1
1 2
1 2
1 1
1 2
1 1
1 2
1 1
1 2
1 9
1 2
1 2
1 1
1 6
1 1
1 1
1 2
1 0
1 0
1 1
1 1
1 0
1 6
1 1
1 0
1 1
1 1
1 0
1 1
1 0
1 0
1 1
1 0
1 2
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 0


In [55]:
targets_val = {}
for key in one_hots_val_processed.keys():
    for k1,v1 in one_hots_val_processed[key].items():
        targets_val[k1] = v1
len(targets_val)

127

In [56]:
targets_val.keys() == all_outputs_val.keys()

True

In [60]:
counts_val = 0
for key in all_outputs_val.keys():
    target = np.argmax(targets_val[key])
    pred = torch.argmax(all_outputs_val[key])
    print(pred.item(),target)
    if target == pred:
        counts_val += 1
print(counts_val / len(targets_val.keys()))

1 0
1 2
1 6
1 4
1 0
1 0
1 0
1 0
1 0
1 4
1 1
1 2
1 0
1 1
1 2
1 2
1 1
1 0
1 0
1 0
1 0
1 0
1 1
1 6
1 0
1 4
1 1
1 2
1 4
1 0
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 6
1 2
1 0
1 0
1 0
1 0
1 1
1 4
1 1
1 0
1 4
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 0
1 2
1 6
1 6
1 1
1 0
1 0
1 4
1 1
1 2
1 6
1 0
1 4
1 1
1 1
1 0
1 0
1 0
1 1
1 4
1 0
1 4
1 2
1 4
1 0
1 0
1 0
1 0
1 1
1 1
1 6
1 1
1 1
1 1
1 1
1 3
1 3
1 4
1 4
1 6
1 3
1 7
1 6
1 1
1 6
1 3
1 3
1 3
1 2
1 2
1 2
1 1
1 1
1 1
1 3
1 2
1 1
1 1
1 6
1 3
1 9
1 1
1 2
1 3
1 3
1 1
1 4
1 3
1 3
0.2677165354330709


In [72]:
list(all_outputs_val.values())[90]

tensor([[0.0037, 0.7112, 0.2655, 0.0034, 0.0032, 0.0029, 0.0029, 0.0031, 0.0014,
         0.0026]])